https://stackoverflow.com/questions/12761510/python-how-can-i-calculate-the-average-word-length-in-a-sentence-using-the-spl

https://pythonprogramming.net/combine-classifier-algorithms-nltk-tutorial/?completed=/sklearn-scikit-learn-nltk-tutorial/

http://streamhacker.com/2012/11/22/text-classification-sentiment-analysis-nltk-scikitlearn/

http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sample-pipeline-for-text-feature-extraction-and-evaluation

http://www.aicbt.com/authorship-attribution/

In [17]:
import pandas as pd
import numpy as np
import re, reprlib, sys, csv, logging, subprocess
import matplotlib.pyplot as plt
from itertools import *
import random as ran

ran.seed(5125)

from pprint import *
from time import *
from rpy2 import *

%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.grid_search import *

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


In [2]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'lexical_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

CalledProcessError: Command '['Rscript', 'lexical_features.R']' returned non-zero exit status 1

In [29]:
data = pd.read_csv("../raw_data/data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=1)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [31]:
data_utf8 = pd.read_csv("../raw_data/data_utf8.csv", parse_dates=True, infer_datetime_format=True)
data_utf8.head(n=1)

,"﻿""#AUTHID""",STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [5]:
word = "likes the sound of thunder."
print(sum(c != ' ' for c in word))
data["Lenght_of_Rows_withSpaces"] = np.nan
data["Lenght_of_Rows_withoutSpaces"] = np.nan
data["Word_Lenght"] = np.nan

23


In [6]:
d = {'n': "no", 'y': "yes"}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

In [7]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66)
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66)
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66)
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66)
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66)

### Grid Search for best parameters

In [53]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

classifierNB = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('nb', MultinomialNB())])

In [11]:
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

In [25]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(neu["STATUS"], neu["cNEU"])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

[Parallel(n_jobs=-1)]: Done  49 tasks       | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 199 tasks       | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 449 tasks       | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 799 tasks       | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 1249 tasks       | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 1799 tasks       | elapsed: 26.8min
[Parallel(n_jobs=-1)]: Done 2449 tasks       | elapsed: 34.9min
[Parallel(n_jobs=-1)]: Done 3199 tasks       | elapsed: 44.8min


Accuracy rbf kernel SVC: 0.582 (+/- 0.01)
Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (1e-05, 1e-06),
 'clf__n_iter': (10, 50, 80),
 'clf__penalty': ('l2', 'elasticnet'),
 'tfidf__norm': ('l1', 'l2'),
 'tfidf__use_idf': (True, False),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__max_features': (None, 5000, 10000, 50000),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 3 folds for each of 1152 candidates, totalling 3456 fits
done in 2911.556s

Best score: 0.607
Best parameters set:
	clf__alpha: 1e-05
	clf__n_iter: 10
	clf__penalty: 'l2'
	tfidf__norm: 'l1'
	tfidf__use_idf: False
	vect__max_df: 1.0
	vect__max_features: None
	vect__ngram_range: (1, 2)


[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed: 48.5min finished


### Lexical and other features

In [28]:
def SyntacticFeatures(chapters):
    """
    Extract feature vector for part of speech frequencies
    """
    list_of_rows_CD = [l.split("\n")[0] for l in chapters]
    def token_to_pos(ch):
        tokens = n.word_tokenize(ch)
        #print(tokens)
        return [p[1] for p in n.pos_tag(tokens)]

    chapters_pos = [token_to_pos(kCD) for kCD in list_of_rows_CD]
    print(chapters_pos)
    pos_list = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS']
    fvs_syntax = np.array([[ch.count(pos) for pos in pos_list]
                           for ch in chapters_pos]).astype(np.float64)
    # normalise by dividing each row by number of tokens in the chapter
    fvs_syntax /= np.c_[np.array([len(ch) for ch in chapters_pos])]
    return fvs_syntax

feature_sets = None
feature_sets = list(SyntacticFeatures(neu["STATUS"]))

NameError: name 'neu' is not defined

In [26]:
def LexicalFeatures(chapters):
    sentence_tokenizer = n.data.load('tokenizers/punkt/english.pickle')
    word_tokenizer = n.tokenize.RegexpTokenizer(r'\w+')
    # create feature vectors
    num_chapters = len(chapters)
    fvs_lexical = np.zeros((len(chapters), 3), np.float64)
    fvs_punct = np.zeros((len(chapters), 3), np.float64)
    for e, ch_text in enumerate(chapters):
        # note: the nltk.word_tokenize includes punctuation
        tokens = nltk.word_tokenize(ch_text.lower())
        words = word_tokenizer.tokenize(ch_text.lower())
        sentences = sentence_tokenizer.tokenize(ch_text)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                       for s in sentences])

        # average number of words per sentence
        fvs_lexical[e, 0] = words_per_sentence.mean()
        # sentence length variation
        fvs_lexical[e, 1] = words_per_sentence.std()
        # Lexical diversity
        fvs_lexical[e, 2] = len(vocab) / float(len(words))

        # Commas per sentence
        fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
        # Semicolons per sentence
        fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
        # Colons per sentence
        fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))

    # apply whitening to decorrelate the features
    fvs_lexical = whiten(fvs_lexical)
    fvs_punct = whiten(fvs_punct)

### Train all new features using Multinomial BN

In [32]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,NBROKERAGE,TRANSITIVITY,StringLength,Number_of_Words,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Avarage_Word_Lenght,Lexical_Diversity
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,0.49,0.1,27,5,1,0,0,0,4.4,0.7


In [97]:
d = {'n': "no", 'y': "yes"}
# http://stackoverflow.com/a/17702781
data_n = data_n.replace(d)
new_fea = data_n[["#AUTHID","STATUS","cCON","cOPN","cEXT","cNEU","cAGR", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
#new_fea.head(n=1)
train_new_fea, test_new_fea = sk.cross_validation.train_test_split(new_fea, train_size = 0.66)
clas_fit_NB = classifierNB.fit(train_new_fea.iloc[1:], train_new_fea[["cCON", "cAGR"]])
predicted_NB = classifierNB.predict(test_new_fea["cCON"])


ValueError: bad input shape (6545, 2)

In [ ]:
scores_NB = cross_validation.cross_val_score(classifierNB, new_fea["STATUS"], new_fea["cCON"], cv = 10)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

### Sentence lenght 

In [ ]:
# df1['e'] = Series(np.random.randn(sLength), index=df1.index) .loc[:,'f']
for index, sentence in data.iterrows():
    filtered = ''.join(filter(lambda x: x not in '".,;!?-', sentence["STATUS"]))
    words = [word for word in filtered.split() if word]
    try:
        avg = sum(map(len, words))/len(words)
    except ZeroDivisionError:
        avg = 0
    data["Word_Lenght"][index] = "%0.1f" % avg